In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
train_path = '../data/train'
valid_path = '../data/valid'
test_path = '../data/test'

In [3]:
target_size = (224, 224)

# If you get a FileNotFoundError, ensure you've run make in this directory.
train_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(train_path, target_size=target_size)
valid_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(valid_path, target_size=target_size)
test_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(test_path, target_size=target_size)

Found 437 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Found 55 images belonging to 2 classes.


In [4]:
index_to_label = {val: key for key, val in train_batches.class_indices.items()}
print(index_to_label)

{0: 'covid', 1: 'normal'}


In [5]:
# Hyperparameters
epochs = 10
learning_rate = 1e-4
dropout_prob_1 = 0
dropout_prob_2 = 0

# Choosing steps per epoch so that 1 epoch equals 1 pass through train/valid/test (steps = samples / batch_size).
train_batch_size = 32
train_steps = train_batches.n // train_batch_size
valid_batch_size = 32
valid_steps = valid_batches.n // valid_batch_size
test_batch_size = 32
test_steps = test_batches.n // test_batch_size

In [6]:
# Early stopping callback
es = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    restore_best_weights=True
)

In [7]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(dropout_prob_1),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(dropout_prob_1),

    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(dropout_prob_1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(dropout_prob_2),
    keras.layers.Dense(1, activation='relu')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [10]:
# %%script echo "Comment the line with %%script echo to run this cell."
history = model.fit(
    train_batches,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=valid_batches,
    validation_steps=valid_steps
)

Epoch 1/10
13/13 [==============================] - 121s 9s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 2/10
13/13 [==============================] - 91s 7s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 3/10
10/13 [======================>.......] - ETA: 33s - loss: 7.7125 - accuracy: 0.5000

KeyboardInterrupt: 

In [ ]:
model.save('arch1.h5')